# Compute and store time window spectra 
Spectra are the power density of $\alpha_{zonal}+i \alpha_{meridional}$ computed with pynsitu.drifters.time_window_processing and pynsitu.tseries.get_spectrum for $\alpha$ among the different positions, velocities or accelerations.
- Spectra  are computed for all trajectories taking time windows of 60 days, with no detrending and hann window by default. For each time windows spectra, the mean positions is also computed. They are store in parquets.

In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd

# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
# import geopandas as gpd
# from shapely.geometry import Polygon

%matplotlib inline
from matplotlib import pyplot as plt

import drifters.utils as ut
import pynsitu as pin
from GDP_lib import root_dir

In [2]:
from dask.distributed import Client

if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    # cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    # cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=5, processes=5, walltime="04:00:00")
    w = cluster.scale(jobs=5)
    # from dask_jobqueue import PBSCluster
    # cluster = PBSCluster()
    # w = cluster.scale(jobs=8) # 5 for gps, 8 for argos
else:
    from dask.distributed import LocalCluster

    cluster = LocalCluster()

client = Client(cluster)
client

/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:237: FutureWarning: extra has been renamed to worker_extra_args. You are still using it (even if only set to []; please also check config files). If you did not set worker_extra_args yet, extra will be respected for now, but it will be removed in a future release. If you already set worker_extra_args, extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.min

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.255:8787/status,
Dashboard: http://10.148.0.255:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.255:36922,Workers: 0
Dashboard: http://10.148.0.255:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Choose key 

In [3]:
key = "argos"  # 'gps' or 'argos'
key2 = "_corrected"  # or '_corrected'

# Load

In [4]:
# LOAD files with velocities and accelerations computed

parquet = os.path.join(root_dir, key + "_av_time" + key2 + ".parquet")
# parquetd = os.path.join(root_dir, key + "_av_time_w.parquet")

# drop all variables but relevant ones
col = [
    "id",
    "time",
    "lon",
    "lat",
    "x",
    "y",
    "ve",
    "vn",
    "ae",
    "an",
    "vex",
    "vny",
    "aex",
    "any",
    "ae_vex",
    "an_vny",
    "vex_diff",
    "vny_diff",
    "err_lon",
    "err_lat",
    "err_ve",
    "err_vn",
]
df = dd.read_parquet(parquet).reset_index()[col].persist()
# dfd = dd.read_parquet(parquetd).reset_index()[col].persist()

In [5]:
def dlonlat_dlonlatm(dlon, dlat, lat):
    """return the error distance in m from the error in degree"""
    R = 6371e3
    # return R * np.arccos(np.sin(lat) ** 2 + (np.cos(lat) ** 2) * np.cos(dlon)), R * dlat
    return R * np.cos(lat) * dlon, R * dlat


def add_errm(df):
    f = dlonlat_dlonlatm(
        df["err_lon"] * np.pi / 180,
        df["err_lat"] * np.pi / 180,
        df["lat"] * np.pi / 180,
    )
    df["err_lonm"] = f[0]
    df["err_latm"] = f[1]


add_errm(df)

In [6]:
df

,id,time,lon,lat,x,y,ve,vn,ae,an,vex,vny,aex,any,ae_vex,an_vny,vex_diff,vny_diff,err_lon,err_lat,err_ve,err_vn,err_lonm,err_latm
npartitions=567,,,,,,,,,,,,,,,,,,,,,,,,
,int64,datetime64[ns],float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


_________________
# Generate Spectra

In [7]:
df = df.rename(columns={"time": "date"})
df["ven2"] = df.ve**2 + df.vn**2

# add time in hours
time_unit = pd.Timedelta("1H")
t_ref = pd.Timestamp(2000, 1, 1)
df["time"] = (df["date"] - t_ref) / time_unit
df

,id,date,lon,lat,x,y,ve,vn,ae,an,vex,vny,aex,any,ae_vex,an_vny,vex_diff,vny_diff,err_lon,err_lat,err_ve,err_vn,err_lonm,err_latm,ven2,time
npartitions=567,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,datetime64[ns],float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
T_str = "60D"  # days, time window given to pin.drifters.time_window_processing
dt = "1H"  # sampling

N = int(pd.Timedelta(T_str) / pd.Timedelta(dt))  # output size
T = pd.Timedelta(T_str) / time_unit  # must be in the same units than time

columns = [
    "lon",
    "lat",
]
columns0 = columns + ["ve", "vn"]
columns1 = columns + ["ae", "an"]
columns2 = columns + ["vex", "vny"]
columns3 = columns + ["aex", "any"]
columns4 = columns + ["vex_diff", "vny_diff"]
columns5 = columns + ["x", "y"]
columns6 = columns + ["err_lon", "err_lat"]
columns7 = columns + ["err_lonm", "err_latm"]
columns8 = columns + ["err_vn", "err_ve"]
columns9 = columns + ["ae_vex", "an_vny"]

labels = [
    "ven",
    "aen",
    "vxy",
    "axy",
    "vxydiff",
    "xy",
    "err_lonlat",
    "err_lonlatm",
    "err_vevn",
    "avxy",
]
Columns = {
    "ven": columns0,
    "aen": columns1,
    "vxy": columns2,
    "axy": columns3,
    "vxydiff": columns4,
    "xy": columns5,
    "err_lonlat": columns6,
    "err_lonlatm": columns7,
    "err_vevn": columns8,
    "avxy": columns9,
}

In [9]:
def process_uv(lon, lat, u, v, N, dt, **kwargs):
    """Wraps spectral calculation: add complex velocity
    Assumes the time series is regularly sampled

    Parameters:
    -----------
        u, v: pd.Series
            zonal, meridional index by time (in days)
        N: int,
            length of the spectrum
        dt: float
            Time sampling in days
        **kwargs:
            passed to mit_equinox.drifters.get_spectrum
    """
    if lon is None:
        uv = None
    else:
        uv = u + 1j * v
    return pin.tseries.get_spectrum(uv, N, dt=dt, **kwargs)

### Compute spectra

In [10]:
# pin.drifters.
group = tuple(df.get_partition(0)["id"].loc[0].values.compute())[0]
dfg = df.groupby("id").get_group(group).compute()

out = pin.drifters.time_window_processing(
    dfg, process_uv, columns0, T, N, id_label="id", dt=dt, geo=True
)

# dfg.set_index("time").plot(x="lon", y="lat")


Df_chunked = {}
for l in Columns:
    df_chunked = (
        df.groupby("id")
        .apply(
            pin.drifters.time_window_processing,
            process_uv,
            Columns[l],
            T,
            N,
            id_label="id",
            dt=dt,
            geo=True,
            meta=out,
        )
        .persist()
    )
    # recompute date
    df_chunked["date"] = t_ref + df_chunked.index * time_unit
    # rename x/y
    df_chunked = df_chunked.rename(columns=dict(x="lon", y="lat"))
    # add nrj
    # df_chunked['ven2'] = df.ve**2 +df.vn**2

    Df_chunked[l] = df_chunked

In [11]:
df_chunked

Dask DataFrame Structure:
                     lon      lat       id      0.0 0.016666666666666666 0.03333333333333333     0.05 0.06666666666666667 0.08333333333333333      0.1 0.11666666666666667 0.13333333333333333     0.15 0.16666666666666666 0.18333333333333332      0.2 0.21666666666666667 0.23333333333333334     0.25 0.26666666666666666 0.2833333333333333      0.3 0.31666666666666665 0.3333333333333333     0.35 0.36666666666666664 0.3833333333333333      0.4 0.4166666666666667 0.43333333333333335     0.45 0.4666666666666667 0.48333333333333334      0.5 0.5166666666666666 0.5333333333333333     0.55 0.5666666666666667 0.5833333333333334      0.6 0.6166666666666667 0.6333333333333333     0.65 0.6666666666666666 0.6833333333333333      0.7 0.7166666666666667 0.7333333333333333     0.75 0.7666666666666666 0.7833333333333333      0.8 0.8166666666666667 0.8333333333333334     0.85 0.8666666666666667 0.8833333333333333      0.9 0.9166666666666666 0.9333333333333333     0.95 0.9666666666666667 0.9833333333333333      1.0 1.0166666666666666 1.0333333333333332     1.05 1.0666666666666667 1.0833333333333333      1.1 1.1166666666666667 1.1333333333333333     1.15 1.1666666666666667 1.1833333333333333      1.2 1.2166666666666666 1.2333333333333334     1.25 1.2666666666666666 1.2833333333333332      1.3 1.3166666666666667 1.3333333333333333     1.35 1.3666666666666667 1.3833333333333333      1.4 1.4166666666666667 1.4333333333333333     1.45 1.4666666666666666 1.4833333333333334      1.5 1.5166666666666666 1.5333333333333332     1.55 1.5666666666666667 1.5833333333333333      1.6 1.6166666666666667 1.6333333333333333     1.65 1.6666666666666667 1.6833333333333333      1.7 1.7166666666666666 1.7333333333333334     1.75 1.7666666666666666 1.7833333333333332      1.8 1.8166666666666667 1.8333333333333333 1.8499999999999999 1.8666666666666667 1.8833333333333333      1.9 1.9166666666666667 1.9333333333333333     1.95 1.9666666666666666 1.9833333333333334      2.0 2.0166666666666666 2.033333333333333     2.05 2.0666666666666664 2.0833333333333335      2.1 2.1166666666666667 2.1333333333333333     2.15 2.1666666666666665 2.183333333333333      2.2 2.216666666666667 2.2333333333333334     2.25 2.2666666666666666 2.283333333333333      2.3 2.3166666666666664 2.3333333333333335     2.35 2.3666666666666667 2.3833333333333333      2.4 2.4166666666666665 2.433333333333333     2.45 2.466666666666667 2.4833333333333334      2.5 2.5166666666666666 2.533333333333333     2.55 2.5666666666666664 2.5833333333333335      2.6 2.6166666666666667 2.6333333333333333     2.65 2.6666666666666665 2.683333333333333      2.7 2.716666666666667 2.7333333333333334     2.75 2.7666666666666666 2.783333333333333      2.8 2.8166666666666664 2.8333333333333335     2.85 2.8666666666666667 2.8833333333333333      2.9 2.9166666666666665 2.933333333333333     2.95 2.966666666666667 2.9833333333333334      3.0 3.0166666666666666 3.033333333333333     3.05 3.0666666666666664 3.0833333333333335      3.1 3.1166666666666667 3.1333333333333333     3.15 3.1666666666666665 3.183333333333333      3.2 3.216666666666667 3.2333333333333334     3.25 3.2666666666666666 3.283333333333333      3.3 3.3166666666666664 3.3333333333333335     3.35 3.3666666666666667 3.3833333333333333      3.4 3.4166666666666665 3.433333333333333 3.4499999999999997 3.466666666666667 3.4833333333333334      3.5 3.5166666666666666 3.533333333333333     3.55 3.5666666666666664 3.5833333333333335      3.6 3.6166666666666667 3.6333333333333333     3.65 3.6666666666666665 3.683333333333333 3.6999999999999997 3.716666666666667 3.7333333333333334     3.75 3.7666666666666666 3.783333333333333      3.8 3.8166666666666664 3.8333333333333335     3.85 3.8666666666666667 3.8833333333333333      3.9 3.9166666666666665 3.933333333333333 3.9499999999999997 3.966666666666667 3.9833333333333334      4.0 4.016666666666667 4.033333333333333     4.05 4.066666666666666 4.083333333333333      4.1 4.116666666666666 4.133333333333333     

## Store spectra

In [12]:
DF = Df_chunked
DFs = {}
for l in DF:
    DFs[l] = DF[l].rename(
        columns={i: str(i) for i in list(DF[l].columns) if isinstance(i, float)}
    )

In [13]:
root_dir = "/home1/datawork/mdemol/GDP"


def store_spectra(Df, root_dir=root_dir):
    for var_key in Columns:
        parquet = os.path.join(
            root_dir, key + "_" + var_key + "_spectra" + key2 + ".parquet"
        )
        df = Df[var_key].repartition(partition_size="100MB").persist()
        df.to_parquet(parquet, engine="pyarrow", overwrite=True)
        print(var_key)


store_spectra(DFs)

ven
aen
vxy
axy
vxydiff
xy
err_lonlat
err_lonlatm
err_vevn
avxy


In [3]:
cluster.close()

/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:237: FutureWarning: extra has been renamed to worker_extra_args. You are still using it (even if only set to []; please also check config files). If you did not set worker_extra_args yet, extra will be respected for now, but it will be removed in a future release. If you already set worker_extra_args, extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.min